# Finetune BART-IT on summarization datasets

- ARTeLab/fanpage
- ARTeLab/ilpost
- **ARTeLab/mlsum-it (this notebook)**


### ARTeLab/mlsum-it



In [1]:
! export COMET_API_KEY=0bfHTco7gsezAkISIiqrE0OPp
! export COMET_PROJECT_NAME=bart_it_mlsum_it
! export COMET_WORKSPACE=morenolq

In [2]:
import comet_ml
import transformers
from datasets import load_dataset

/home/mlaquatra/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("bart-it-s/")
tokenizer = transformers.AutoTokenizer.from_pretrained("tokenizer_bart_it")

In [6]:
dataset = load_dataset("ARTeLab/mlsum_it")
train_dataset = dataset["train"]
test_dataset = dataset["test"]
val_dataset = dataset["validation"]

print ("Len train: ", len(train_dataset))
print ("Len test: ", len(test_dataset))
print ("Len val: ", len(val_dataset))


# average length input in training set
avg_len_train = sum([len(tokenizer.encode(x)) for x in train_dataset["source"]]) / len(train_dataset)
# average length output in training set
avg_len_output_train = sum([len(tokenizer.encode(x)) for x in train_dataset["target"]]) / len(train_dataset)

# average length input in test set
avg_len_test = sum([len(tokenizer.encode(x)) for x in test_dataset["source"]]) / len(test_dataset)
# average length output in test set
avg_len_output_test = sum([len(tokenizer.encode(x)) for x in test_dataset["target"]]) / len(test_dataset)

# average length input in validation set
avg_len_val = sum([len(tokenizer.encode(x)) for x in val_dataset["source"]]) / len(val_dataset)
# average length output in validation set
avg_len_output_val = sum([len(tokenizer.encode(x)) for x in val_dataset["target"]]) / len(val_dataset)



train_input = train_dataset["source"]
train_target = train_dataset["target"]

test_input = test_dataset["source"]
test_target = test_dataset["target"]

val_input = val_dataset["source"]
val_target = val_dataset["target"]

Using custom data configuration ARTeLab--fanpage-3134261830d8de61
Found cached dataset csv (/home/mlaquatra/.cache/huggingface/datasets/ARTeLab___csv/ARTeLab--fanpage-3134261830d8de61/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 3/3 [00:00<00:00, 564.79it/s]


Len train:  67492
Len test:  8437
Len val:  8436
Len train: 67492 - Avg len input: 443.40555917738396 - Avg len output: 63.170509097374506
Len test: 8437 - Avg len input: 442.9748725850421 - Avg len output: 63.22425032594524
Len val: 8436 - Avg len input: 444.57622095779993 - Avg len output: 63.580606922712185


In [10]:
print (f"Len train: {len(train_dataset)} - Avg len input: {avg_len_train} - Avg len output: {avg_len_output_train}")
print (f"Len test: {len(test_dataset)} - Avg len input: {avg_len_test} - Avg len output: {avg_len_output_test}")
print (f"Len val: {len(val_dataset)} - Avg len input: {avg_len_val} - Avg len output: {avg_len_output_val}")

Len train: 67492 - Avg len input: 443.40555917738396 - Avg len output: 63.170509097374506
Len test: 8437 - Avg len input: 442.9748725850421 - Avg len output: 63.22425032594524
Len val: 8436 - Avg len input: 444.57622095779993 - Avg len output: 63.580606922712185


In [7]:
from typing import List

import torch
import transformers


class Dataset(torch.utils.data.Dataset):
    """
    This class is inteded for sequence classification tasks.
    :param source_text: List of source text that is used as input to the model.
    :param target_text: List of target text that is used as expected output from the model.
    :param tokenizer: The tokenizer to be used for tokenizing the texts. It can be an instance of the transformers AutoTokenizer class or a custom tokenizer.
    :param max_input_length: The maximum length of the tokenized input text.
    :param max_output_length: The maximum length of the tokenized output text.
    :param padding: The padding strategy to be used. Available options are available in the transformers library.
    :param truncation: Whether to truncate the text or not.
    """

    def __init__(
        self,
        source_text: List[str],
        target_text: List[str],
        tokenizer,
        max_input_length: int = 1024,
        max_output_length: int = 128,
        padding: str = "max_length",
        truncation: bool = True,
    ):

        self.source_text = source_text
        self.target_text = target_text
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length
        self.tokenizer = tokenizer
        self.padding = padding
        self.truncation = truncation

    def __getitem__(self, idx):
        """
        This function is called to get the tokenized source and target text for a given index.
        :param idx: The index of the text and label to be returned.
        :return: A dictionary containing the tokenized source (`input_ids`) with attention mask (`attention_mask`) and the tokenized target (`labels`).
        """
        input = self.tokenizer(
            self.source_text[idx],
            max_length=self.max_input_length,
            padding=self.padding,
            truncation=self.truncation,
            return_tensors="pt",
        )

        output = self.tokenizer(
            text_target = self.target_text[idx],
            max_length=self.max_output_length,
            padding=self.padding,
            truncation=self.truncation,
            return_tensors="pt",
        )

        item = {
            "input_ids": input["input_ids"].squeeze(),
            "attention_mask": input["attention_mask"].squeeze(),
            "labels": output["input_ids"].squeeze(),
        }

        return item

    def __len__(self):
        return len(self.source_text)

train_dataset = Dataset(source_text=train_input, target_text=train_target, tokenizer=tokenizer)
test_dataset = Dataset(source_text=test_input, target_text=test_target, tokenizer=tokenizer)
val_dataset = Dataset(source_text=val_input, target_text=val_target, tokenizer=tokenizer)

In [8]:
CHECKPOINT_DIR = "mlsum_it_bart_it"
EPOCHS = 10
BATCH_SIZE = 32
LOG_DIR = "mlsum_it_bart_it_logs"
LOGGING_STEPS = 100
LEARNING_RATE = 1e-5
DATALOADER_NUM_WORKERS = 4
SAVE_TOTAL_LIMIT = 5
USE_CUDA = True
FP16 = True
HUB_MODEL_ID = ""
PUSH_TO_HUB = False

In [11]:
training_arguments = transformers.TrainingArguments(
    output_dir=CHECKPOINT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=LOG_DIR,
    logging_steps=LOGGING_STEPS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=LEARNING_RATE,
    dataloader_num_workers=DATALOADER_NUM_WORKERS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    no_cuda=not (USE_CUDA),
    fp16=FP16,
    metric_for_best_model="R2",
    greater_is_better=True,
    hub_model_id=HUB_MODEL_ID,
    push_to_hub=PUSH_TO_HUB,
)

In [17]:
import evaluate

rouge = evaluate.load("rouge")


def compute_metrics(pred):

    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str,
        references=label_str,
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
    )

    return {
        "R1": round(rouge_output["rouge1"], 4),
        "R2": round(rouge_output["rouge2"], 4),
        "RL": round(rouge_output["rougeL"], 4),
        "RLsum": round(rouge_output["rougeLsum"], 4),
    }

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

trainer = transformers.Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

trainer.train()

Using cuda_amp half precision backend
/home/mlaquatra/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67492
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 10550


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/mlaquatra/miniconda3/envs/nlp/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,R1,R2,Rl,Rlsum
1,1.004700,1.000556,0.526800,0.269200,0.487400,0.487400
2,0.981000,0.980523,0.531600,0.274100,0.492700,0.492800
3,0.953100,0.971745,0.534300,0.277400,0.495400,0.495400
4,0.935300,0.964408,0.536600,0.279600,0.498400,0.498300
5,0.912200,0.962281,0.536800,0.280500,0.498700,0.498700
6,0.904000,0.959691,0.537800,0.281900,0.499900,0.499900
7,0.899100,0.958759,0.538100,0.282700,0.500400,0.500300
8,0.886400,0.957292,0.538300,0.282700,0.500600,0.500600
9,0.876000,0.956439,0.538600,0.283000,0.501000,0.500900
10,0.872400,0.956755,0.538700,0.282800,0.501200,0.501100


	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


***** Running Evaluation *****
  Num examples = 8436
  Batch size = 64


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Saving model checkpoint to fanpage_bart_it/checkpoint-10550
Configuration saved in fanpage_bart_it/checkpoint-10550/config.json
Model weights saved in fanpage_bart_it/checkpoint-10550/pytorch_model.bin
Deleting older checkpoint [fanpage_bart_it/checkpoint-5275] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from fanpage_bart_it/checkpoint-9495 (score: 0.283).


TrainOutput(global_step=10550, training_loss=0.9340590606255554, metrics={'train_runtime': 8637.8613, 'train_samples_per_second': 78.135, 'train_steps_per_second': 1.221, 'total_flos': 4.115233303953408e+17, 'train_loss': 0.9340590606255554, 'epoch': 10.0})

In [18]:
print (trainer.evaluate(test_dataset))

***** Running Evaluation *****
  Num examples = 8437
  Batch size = 64


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/mlaquatra/miniconda3/envs/nlp/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{'eval_loss': 0.9436779618263245, 'eval_R1': 0.5396, 'eval_R2': 0.2838, 'eval_RL': 0.5023, 'eval_RLsum': 0.5024, 'eval_runtime': 57.3394, 'eval_samples_per_second': 147.141, 'eval_steps_per_second': 2.302, 'epoch': 10.0}
